In [4]:
fun = lambda x: x ** 2

In [10]:
import numdifftools as nd

first = nd.Derivative(fun)
print first(3)

second = nd.Derivative(fun, n = 2)
print second(3)

third = nd.Derivative(fun, n = 3)
print third(3)

6.0
2.0
-1.05435936153e-13


In [15]:
from scipy.misc import derivative

print derivative(fun, 3, n = 1)
print derivative(fun, 3, n = 2)
# print derivative(fun, 3, n = 3)

6.0
2.0


In [26]:
from scipy.misc import derivative
import numpy as np

def hess(f, x):
    x = np.array(x).astype(float)
    h = 0.00001
    hess = np.zeros_like(x)
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    def partial_derivative(func, var=0, point=[]):
        args = point[:]
        def wraps(x):
            args[var] = x
            return func(*args)
        return derivative(wraps, point[var], dx = 1e-6)
        
    while not it.finished:
        ix = it.multi_index
        print hess
        print ix
        print hess[ix]
        # hess[ix] = partial_derivative(f, ix, x[ix])
        it.iternext()
        
    return hess

hess(foo, [3, 1])

[ 0.  0.]
(0,)
0.0
[ 0.  0.]
(1,)
0.0


array([ 0.,  0.])

In [21]:
foo = lambda x, y: x ** 2 + y ** 3

from scipy.misc import derivative
derivative(foo, 1, dx = 1e-6, args = (3, ))

def partial_derivative(func, var=0, point=[]):
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = 1e-6)

partial_derivative(foo, 1, [3, 1])

2.9999999995311555